In [242]:
# import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [243]:
df = pd.read_csv(r"C:\Users\Somesh\Downloads\archive (1)\fraudTrain.csv")

In [244]:
df

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [245]:
# Display basic information about the dataset
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [246]:
# Handle missing values (if any)
data = df.dropna()

In [247]:
# Identify categorical columns that need encoding
categorical_columns = ['category', 'gender', 'merchant', 'city', 'state','first','last','street','job','dob','trans_num']

In [248]:


# Apply Label Encoding to categorical columns
le = LabelEncoder()
for col in categorical_columns:
    data[col] = le.fit_transform(data[col])


In [249]:
# Convert date columns to datetime and extract useful features
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data['year'] = data['trans_date_trans_time'].dt.year
data['month'] = data['trans_date_trans_time'].dt.month
data['day'] = data['trans_date_trans_time'].dt.day
data['hour'] = data['trans_date_trans_time'].dt.hour


In [250]:
# Drop the original datetime column
data = data.drop(['trans_date_trans_time'], axis=1)

# Define the target variable and features
X_train = data.drop(['is_fraud'], axis=1)
y_train = data['is_fraud']


In [251]:
X_train

,Unnamed: 0,cc_num,merchant,category,amt,first,last,gender,street,city,...,job,dob,trans_num,unix_time,merch_lat,merch_long,year,month,day,hour
0,0,2703186189652095,514,8,4.97,162,18,0,568,526,...,370,779,56438,1325376018,36.011293,-82.048315,2019,1,1,0
1,1,630423337322,241,4,107.23,309,157,0,435,612,...,428,607,159395,1325376044,49.159047,-118.186462,2019,1,1,0
2,2,38859492057661,390,0,220.11,115,381,1,602,468,...,307,302,818703,1325376051,43.150704,-112.154481,2019,1,1,0
3,3,3534093764340240,360,2,45.00,163,463,1,930,84,...,328,397,544575,1325376076,47.034331,-112.561071,2019,1,1,0
4,4,375534208663984,297,9,41.96,336,149,1,418,216,...,116,734,831111,1325376186,38.674999,-78.632459,2019,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,30263540414123,499,0,15.56,121,332,1,154,330,...,215,298,344658,1371816728,36.841266,-111.690765,2020,6,21,12
1296671,1296671,6011149206456997,2,1,51.70,160,463,1,856,813,...,360,630,199896,1371816739,38.906881,-78.246528,2020,6,21,12
1296672,1296672,3514865930894695,599,1,105.93,74,67,1,158,346,...,308,412,366013,1371816752,33.619513,-105.130529,2020,6,21,12
1296673,1296673,2720012583106919,509,1,74.90,179,304,1,433,471,...,485,639,1086299,1371816816,42.788940,-103.241160,2020,6,21,12


In [252]:
# Identify categorical columns that need encoding
categorical_columns = ['category', 'gender']  # Add other string columns if any

In [253]:
# Normalize numerical features
scaler = StandardScaler()
numerical_columns = ['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long']
X_train[numerical_columns] = scaler.fit_transform(X[numerical_columns])

In [254]:
df_test = pd.read_csv(r"archive (1)\fraudTest.csv")

In [255]:
df_test

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [51]:
# df_test = df_test.dropna(inplace=True)

In [256]:
categorical_columns = ['category', 'gender', 'merchant', 'city', 'state','first','last','street','job','dob','trans_num']

In [257]:
df_test

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [229]:
df_test[df_test['first']== "Jennifer"]

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
107,107,2020-06-21 12:47:21,4191109180926792,fraud_Gottlieb-Hansen,personal_care,12.64,Jennifer,Spencer,F,477 Wheeler Burg,...,39.6251,-82.7552,4512,Farm manager,1992-10-03,36889cdd6d799ccda86cf3c6509010b1,1371818841,39.666410,-83.562243,0
136,136,2020-06-21 12:57:44,630424987505,fraud_Kling-Ernser,entertainment,19.30,Jennifer,Black,F,2870 Bean Terrace Apt. 756,...,39.1505,-79.5030,836,Forensic psychologist,1981-08-29,3c16826f4605f06be65c218e14321e10,1371819464,39.097309,-79.398315,0
269,269,2020-06-21 13:46:16,4010002218955876,fraud_Mayert Group,shopping_pos,13.13,Jennifer,Bishop,F,589 Matthew Brooks,...,29.7972,-95.3288,2906700,Animator,1983-08-24,2493f6e4ff6001a41028dcd3d9b2a145,1371822376,29.513604,-95.923421,0
290,290,2020-06-21 13:52:27,4767265376804500,fraud_Ziemann-Waters,health_fitness,196.93,Jennifer,Conner,F,4655 David Island,...,40.3750,-75.2045,2158,Transport planner,1961-06-19,d880af591bf9096498fa4a46a897c6d3,1371822747,39.502890,-74.598794,0
311,311,2020-06-21 14:00:11,4191109180926792,"fraud_Bahringer, Bergnaum and Quitzon",home,117.84,Jennifer,Spencer,F,477 Wheeler Burg,...,39.6251,-82.7552,4512,Farm manager,1992-10-03,8897077856aef4eafd181e856606c5e2,1371823211,39.253923,-83.606873,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554882,554882,2020-12-31 19:07:03,630424987505,fraud_Botsford and Sons,home,24.54,Jennifer,Black,F,2870 Bean Terrace Apt. 756,...,39.1505,-79.5030,836,Forensic psychologist,1981-08-29,2493a60376087c11f5c848de58c212d0,1388516823,39.241078,-79.651990,0
555113,555113,2020-12-31 20:28:27,6011679934075347,fraud_Hills-Boyer,personal_care,57.48,Jennifer,Gonzalez,F,5517 Stacy Land,...,41.0539,-106.0763,100,Public librarian,1974-04-16,3596a307a60b16e04bcc15d9d0478837,1388521707,41.025590,-105.941465,0
555218,555218,2020-12-31 21:06:01,4191109180926792,"fraud_Kihn, Abernathy and Douglas",shopping_net,6.41,Jennifer,Spencer,F,477 Wheeler Burg,...,39.6251,-82.7552,4512,Farm manager,1992-10-03,1d69629648674cae7ea421079fcec62a,1388523961,40.340281,-82.908047,0
555357,555357,2020-12-31 21:51:05,4277232699798846,"fraud_Roberts, Ryan and Smith",personal_care,77.60,Jennifer,Vance,F,13299 Patrick Terrace,...,46.1664,-103.7079,475,Illustrator,1955-11-10,a8fdd703e3d111329129dba8f6dfc980,1388526665,45.581315,-103.125850,0


In [258]:
# Apply Label Encoding to categorical columns
le = LabelEncoder()
for col in categorical_columns:
    df_test[col] = le.fit_transform(df_test[col])


In [259]:
# Convert date columns to datetime and extract useful features
df_test['trans_date_trans_time'] = pd.to_datetime(df_test['trans_date_trans_time'])
df_test['year'] = df_test['trans_date_trans_time'].dt.year
df_test['month'] = df_test['trans_date_trans_time'].dt.month
df_test['day'] = df_test['trans_date_trans_time'].dt.day
df_test['hour'] = df_test['trans_date_trans_time'].dt.hour

In [260]:
# Drop the original datetime column
df_test = df_test.drop(['trans_date_trans_time'], axis=1)

# Define the target variable and features
X_test = df_test.drop(['is_fraud'], axis=1)
y_test = df_test['is_fraud']


In [234]:
X_test

,Unnamed: 0,cc_num,merchant,category,amt,first,last,gender,street,city,...,job,dob,trans_num,unix_time,merch_lat,merch_long,year,month,day,hour
0,0,2291163933867244,319,10,2.86,151,115,1,341,157,...,275,376,98699,1371816865,33.986391,-81.200714,2020,6,21,12
1,1,3573030041201292,591,10,29.84,163,457,0,354,16,...,392,760,108785,1371816873,39.450498,-109.960431,2020,6,21,12
2,2,3598215285024754,611,5,41.28,24,249,0,865,61,...,259,421,433979,1371816893,40.495810,-74.196111,2020,6,21,12
3,3,3591919803438423,222,9,60.05,42,457,1,320,764,...,407,718,71993,1371816915,28.812398,-80.883061,2020,6,21,12
4,4,3526826139003047,292,13,3.19,247,261,1,548,247,...,196,177,190585,1371816917,44.959148,-85.884734,2020,6,21,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,30560609640617,507,5,43.77,235,315,1,531,443,...,460,334,336391,1388534347,39.946837,-91.333331,2020,12,31,23
555715,555715,3556613125071656,264,7,111.84,171,424,1,540,401,...,198,890,70372,1388534349,29.661049,-96.186633,2020,12,31,23
555716,555716,6011724471098086,496,7,86.88,18,239,0,126,104,...,294,609,235088,1388534355,46.658340,-119.715054,2020,12,31,23
555717,555717,4079773899158,75,13,7.99,111,342,1,663,476,...,58,331,43546,1388534364,44.470525,-117.080888,2020,12,31,23


In [236]:
# Identify categorical columns that need encoding
categorical_columns = ['category', 'gender']  # Add other string columns if any

In [261]:
# Normalize numerical features
scaler = StandardScaler()
numerical_columns = ['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long']
X_test[numerical_columns] = scaler.fit_transform(X_test[numerical_columns])

In [262]:
# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
print("Logistic Regression Results")
print(classification_report(y_test, lr_preds))

Logistic Regression Results
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.00      0.00      0.00      2145

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       0.99      1.00      0.99    555719



C:\Users\Somesh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Somesh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Somesh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classific

In [239]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_preds = dt.predict(X_test)
print("Decision Tree Results")
print(classification_report(y_test, dt_preds))

Decision Tree Results
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.51      0.76      0.61      2145

    accuracy                           1.00    555719
   macro avg       0.76      0.88      0.81    555719
weighted avg       1.00      1.00      1.00    555719



In [240]:
# Random Forest Classifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("Random Forest Results")
print(classification_report(y_test, rf_preds))

Random Forest Results
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.93      0.66      0.77      2145

    accuracy                           1.00    555719
   macro avg       0.96      0.83      0.89    555719
weighted avg       1.00      1.00      1.00    555719



In [241]:
# Compare model performance
print("Accuracy Scores:")
print("Logistic Regression:", accuracy_score(y_test, lr_preds))
print("Decision Tree:", accuracy_score(y_test, dt_preds))
print("Random Forest:", accuracy_score(y_test, rf_preds))

Accuracy Scores:
Logistic Regression: 0.9961401355721147
Decision Tree: 0.9963020879257323
Random Forest: 0.9984992415231438
